In [179]:
import os 
import pandas as pd 
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns 
import plotly.express as px 
import json
plt.style.use('ggplot')
pd.set_option('display.max_columns',200)
pd.set_option('display.max_colwidth', None)

In [4]:
data_path = '/Users/macintosh/desktop/dataset/airbnb_bangkok_data/'

In [165]:
calendar_path = data_path+'calendar.csv'
listing_path = data_path+'listings.csv'
listing2_path = data_path+'listings2.csv'
neighbourhoods_path = data_path + 'neighbourhoods.csv'
neighbourhoods_json_path = data_path + 'neighbourhoods.geojson' # not use for now, we will revisit if have any idea from geo.
reviews_path = data_path + 'reviews.csv'
reviews2_path = data_path + 'reviews2.csv'
calendar_28_dec_22_path = data_path+'28_dec_22_calendar.csv'


In [23]:
df_calendar = pd.read_csv(calendar_path)
df_listing = pd.read_csv(listing_path)
df_listing2 = pd.read_csv(listing2_path)
df_neighbourhoods = pd.read_csv(neighbourhoods_path)
df_reviews = pd.read_csv(reviews_path)
df_reviews2 = pd.read_csv(reviews2_path)

In [167]:
df_cal_28dec22 = pd.read_csv(calendar_28_dec_22_path)
df_cal_28dec22.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,27934,2022-12-28,f,"$1,905.00","$1,905.00",3,90
1,27934,2022-12-29,f,"$1,905.00","$1,905.00",3,90
2,27934,2022-12-30,f,"$1,905.00","$1,905.00",3,90
3,27934,2022-12-31,f,"$1,905.00","$1,905.00",3,90
4,27934,2023-01-01,f,"$1,905.00","$1,905.00",3,90


In [148]:
df_listing2.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,30216638,https://www.airbnb.com/rooms/30216638,20230328184442,2023-03-29,previous scrape,Ama Hostel Bangkok - Private Room For 8 Person,Our mixed dormitory bed is comfortable and cle...,"Ama Hostel Bangkok is set in Bangkok, 700 m fr...",https://a0.muscache.com/pictures/2856e97a-827e...,226941867,https://www.airbnb.com/users/show/226941867,Nakun,2018-11-22,NaN,NaN,NaN,NaN,0%,f,https://a0.muscache.com/im/pictures/user/19e86...,https://a0.muscache.com/im/pictures/user/19e86...,Chinatown,8,8,"['email', 'phone']",t,t,"Khet Samphanthawong, Krung Thep Maha Nakhon, T...",Samphanthawong,NaN,13.74188,100.50202,Shared room in hostel,Shared room,8,NaN,3 shared baths,1.0,8.0,"[""Hair dryer"", ""Essentials"", ""Wifi"", ""Air cond...","$4,800.00",1,365,1,1,365,365,1.0,365.0,NaN,t,0,0,0,0,2023-03-29,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,8,0,7,1,NaN
1,30387182,https://www.airbnb.com/rooms/30387182,20230328184442,2023-03-29,previous scrape,Ama Hostel Bangkok - Quadruple Room Shared Bath,"Experience Culture, Street Food, Heritage Thai...","Ama Hostel Bangkok is set in Bangkok, 700 m fr...",https://a0.muscache.com/pictures/9777182f-7b09...,226941867,https://www.airbnb.com/users/show/226941867,Nakun,2018-11-22,NaN,NaN,NaN,NaN,0%,f,https://a0.muscache.com/im/pictures/user/19e86...,https://a0.muscache.com/im/pictures/user/19e86...,Chinatown,8,8,"['email', 'phone']",t,t,"Khet Samphanthawong, Krung Thep Maha Nakhon, T...",Samphanthawong,NaN,13.74182,100.50257,Private room in hostel,Private room,4,NaN,3 shared baths,1.0,4.0,"[""TV"", ""Essentials"", ""Wifi"", ""Air conditioning...","$2,800.00",1,365,1,1,365,365,1.0,365.0,NaN,t,0,0,0,0,2023-03-29,1,0,0,2020-01-01,2020-01-01,5.0,5.0,5.0,5.0,4.0,4.0,5.0,NaN,t,8,0,7,1,0.03
2,35190417,https://www.airbnb.com/rooms/35190417,20230328184442,2023-03-29,previous scrape,A Bit of Heaven in Bangkok,Luxurious Condo is in Supalai Wellington on th...,A wide variety of art and cultural shows and p...,https://a0.muscache.com/pictures/efde2ee7-bbb6...,100026163,https://www.airbnb.com/users/show/100026163,Wilts,2016-10-17,NaN,NaN,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/763b3...,https://a0.muscache.com/im/pictures/user/763b3...,NaN,1,1,"['email', 'phone']",t,f,"Khet Huai Khwang, Krung Thep Maha Nakhon, Thai...",Huai Khwang,NaN,13.76578,100.58413,Entire condo,Entire home/apt,4,NaN,3 baths,3.0,3.0,"[""TV"", ""Essentials"", ""Kitchen"", ""Hot tub"", ""Wi...","$2,570.00",30,365,30,30,1125,1125,30.0,1125.0,NaN,f,0,0,0,0,2023-03-29,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
3,31450489,https://www.airbnb.com/rooms/31450489,20230328184442,2023-03-29,previous scrape,Warm two-bedroom,"The s

In [176]:
df_calendar.date.agg(['min', 'max'])

min    2023-03-28
max    2024-03-27
Name: date, dtype: object

In [177]:
df_cal_28dec22.date.agg(['min', 'max'])

min    2022-12-28
max    2024-01-02
Name: date, dtype: object

In [180]:
df_reviews2.head()
# df_calendar.query('listing_id == 39020187').price.unique()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,27934,1094339,2012-04-07,1368195,Michael,"We stayed in the apartment for a week and we enjoyed it very much. Nuttee is a very nice host, and she did her best to accommodate us. Everything is perfect in the apartment, and we love the view from the balcony. The apartment is very modern and spacious, and the location is very central, 10 mins walk to BTS station (and supermarket), or 10 mins by bus/taxi to central world shopping mall. There are also a lot of food stalls and massage nearby. We will definitely stay there again for our next visit to Bangkok. Highly recommended."
1,27934,1241042,2012-05-07,2007324,Scott,"My girlfriend and I recently stayed in Nuttee's condo for a month. It is a beautiful condo, with a great view, in a great location near Victory Monument. There is an abundance of wonderful street food right around the corner, as well as shopping. This is a perfect area from which to base exploration of Bangkok.\r<br/>\r<br/>Nuttee was out of town, but was still very accessible via email. Her assistant, Suchart, was great, meeting us when we arrived, arranging cleaning of the condo, and seeing us off when we left. He did everything possible to make certain our stay was perfect.\r<br/>\r<br/>We could not have asked for a better experience, and hope to stay here again when we return to Bangkok."
2,27934,1523384,2012-06-20,2263352,Marc,"I stayed for one month at the condo and was realy pleased. \r<br/>The condo is at the 19th floor, quiet, modern and clean, and you realy have a ""great city view"" by day and much more by night. The 24 h Security is very friendly and there is a magnetic entrance system. So you feel save.\r<br/>\r<br/>The communication between Nuttee, her assistant and me was perfect and it was no problem to find the building. The assistant of Nuttee was very friendly and explained me everything.\r<br/>\r<br/>There is a good and cheap laundry at Rachaprarob road (4 min walking), 7eleven also and on the way to BTS Victory Monument some nice and cheap restaurants and pubs (Thanon Rang Nam Rd).\r<br/>\r<br/>The location is not direct in the tourist-center of the city but you can see and feel the real Bangkok life as a good starting point to explore the city. Taxi is so cheap that the location is not really important. \r<br/>\r<br/>I would give 9 of 10 points. Recommendable. ;-)\r<br/>...................\r<br/>\r<br/>My special tip: if you like Jazz/Blues live music visit the saxophone pub and restaurant near Victory Monument. Perfect music, great vibe !!!"
3,27934,1655571,2012-07-08,558987,Leyla,"Nuttee was a great host! I really enjoyed her apartment and she was absolutely lovely! She even had a bowl of fresh fruit and a sim card for me when I arrived. \r<br/>The apartment itself is really spacious for a single person or couple and has all the mod cons needed to have a relaxing stay in Bangkok. Great air con and the swimming pool was refreshing after a long humid day walking around!\r<br/>I would recommend the apartment as a great, comfortable and relaxing place to stay in Bangkok for sure. \r<br/>The only thing that you should keep in mind is that the BTS (sky train) is a 10 minute walk away - you can take a motorcycle taxi for a few baht or walking is fine in the mornings. There are nice cafes and food places along the street that you walk down to the BTS so its not that bad unless its the middle of a hot day!!"
4,27934,1972192,2012-08-13,2359865,Rachel,"Nuttee was an amazing host. She and her daughter waited for us at the lobby and gave us a quick rundown of the place. It's a studio apartment, with a classy partition dividing the bedroom from the living room -- and the view from the balcony is awesome. Bed was huge, but if you're coming in a group of 3, the sofa may not always be the most comfortable place for the last guy (bed IS huge enough for three to sleep in though). Shower pressure in the main bathroom was a little low, but we could deal with 

In [75]:
df_listing.price.agg(['max', 'min'])

max    1000000
min        120
Name: price, dtype: int64

In [178]:
# Assumption 
# 1. chance room will booked in level date can compare with last year(let's say today for last year it booked or not?)
# 2. feature that relavant with season or holiday?
# 3. long weekend can we compute?
# 4. maybe depending on specific day that make guests decision to booked or not?
# 5. can we find demand on each day?
# 6. cumulative sum holiday?
# 7. zone impact with booking --> good zone will have hight chance to booked first